In [1]:
# change this to the right path if you use a different path
# or specify the ORCA_PATH environmental variable
import os
import sys
if os.getenv("ORCA_PATH"):
    ORCA_PATH = os.getenv("ORCA_PATH")
else:
    ORCA_PATH = "../orca"
sys.path.append(ORCA_PATH)

In [2]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import orca_predict
orca_predict.load_resources(models=[])
from orca_predict import *

In [4]:
from orca_models import HCTnoc
hctnoc =HCTnoc()
hctnoc.eval()
hctnoc.cuda()


HCTnoc(
  (net): DataParallel(
    (module): Encoder2(
      (lblocks): ModuleList(
        (0): Sequential(
          (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
          (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
          (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
          (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
          (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_runn

In [5]:
target_hctnoc = Genomic2DFeatures(
    [ORCA_PATH+"/resources/4DNFILP99QJS.HCT_auxin6h.rebinned.mcool::/resolutions/4000"],
    ["r4000"],
    (8000, 8000),
    cg=True,
)

In [6]:
import torch
with torch.no_grad():
    mallpreds = []
    malltargets = []
    malltargetnans = []
    mallstarts = []
    for chrm in ['chr9','chr10']:
        chrlen = [len for c, len in hg38.get_chr_lens() if c == chrm][0]
        for pos in np.arange(16000000, chrlen-16000000, 500000):
            print(pos)
            twotargets = [torch.FloatTensor(target_hctnoc.get_feature_data(chrm,pos-16000000, pos+16000000)[None,:,:])]


            sequence = hg38.get_encoding_from_coords(chrm, pos-16000000, pos+16000000)
            seqs = [torch.FloatTensor(sequence[None,:,:]), torch.FloatTensor(sequence[None,::-1,::-1].copy())]

            allpreds = []
            alltargets = []
            alltargetnans = []
            allstarts = []

            for iii, seq in enumerate(seqs):
                for ii, model in enumerate([hctnoc]):

                    target = twotargets[ii]
                    encoding1, encoding2, encoding4, encoding8, encoding16, encoding32 = model.net(model.net0(torch.Tensor(seq.float()).transpose(1,2).cuda()))

                    encodings = {1: encoding1, 2: encoding2, 4: encoding4, 8:encoding8, 16:encoding16, 32:encoding32}

                    def eval_step(level, encoding, start, coarse_pred=None):
                        distenc=torch.log(torch.FloatTensor(model.normmats[level][None,None,:,:]).cuda()).expand(seq.shape[0],1,250,250)
                        if coarse_pred is not None:
                            pred = model.denets[level].forward(encoding[:,:,int(start/level):int(start/level)+250], distenc, coarse_pred)
                        else:
                            pred = model.denets[level].forward(encoding[:,:,int(start/level):int(start/level)+250], distenc)


                        return pred

                    def t_step(level, start):
                        target_r = np.nanmean(np.nanmean(np.reshape(target[:,start:start+250*level,start:start+250*level].numpy(),(target.shape[0],250,level,250,level)),axis=4),axis=2)
                        target_np = np.log((target_r+model.epss[level])/(model.normmats[level]+model.epss[level]))[:,0:,0:].reshape((target_r.shape[0],-1))
                        target_nan = np.mean(np.mean(np.isnan(np.reshape(target[:,start:start+250*level,start:start+250*level].numpy(),(target.shape[0],250,level,250,level))),axis=4),axis=2)
                        return target_np, target_nan

                    start32 = 0
                    pred32 = eval_step(32, encodings[32], start32)
                    target32, target32nan = t_step(32, start32)
                    start16 = start32+(62+iii)*32
                    pred16 = eval_step(16, encodings[16], start16, pred32[:,:,(62+iii):(187+iii),(62+iii):(187+iii)].detach())
                    target16, target16nan = t_step(16, start16)
                    start8 = start16+(62+iii)*16
                    pred8 = eval_step(8, encodings[8], start8, pred16[:,:,(62+iii):(187+iii),(62+iii):(187+iii)].detach())
                    target8, target8nan = t_step(8, start8)
                    start4 = start8+(62+iii)*8
                    pred4 = eval_step(4, encodings[4], start4, pred8[:,:,(62+iii):(187+iii),(62+iii):(187+iii)].detach())
                    target4, target4nan = t_step(4, start4)
                    start2 = start4+(62+iii)*4
                    pred2 = eval_step(2, encodings[2], start2, pred4[:,:,(62+iii):(187+iii),(62+iii):(187+iii)].detach())
                    target2, target2nan = t_step(2, start2)
                    start1 = start2+(62+iii)*2
                    pred1 = eval_step(1, encodings[1], start1, pred2[:,:,(62+iii):(187+iii),(62+iii):(187+iii)].detach())
                    target1, target1nan = t_step(1, start1)

                    allpreds.append([pred1, pred2, pred4, pred8, pred16, pred32])
                    alltargets.append([target1, target2, target4, target8, target16, target32])
                    alltargetnans.append([target1nan, target2nan, target4nan, target8nan, target16nan, target32nan])
                    allstarts.append([start1, start2, start4, start8, start16, start32 ])
            mallpreds.append(allpreds)
            malltargets.append(alltargets)
            malltargetnans.append(alltargetnans)
            mallstarts.append(allstarts)

16000000


<PYTHONPATH>/lib/python3.7/site-packages/cooltools/lib/numutils.py:1317: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


16500000
17000000
17500000
18000000
18500000
19000000
19500000
20000000
20500000
21000000
21500000
22000000
22500000
23000000
23500000
24000000
24500000
25000000
25500000
26000000
26500000
27000000
27500000
28000000
28500000
29000000
29500000
30000000
30500000
31000000
31500000
32000000
32500000
33000000
33500000
34000000
34500000
35000000
35500000
36000000
36500000
37000000
37500000
38000000
38500000
39000000
39500000
40000000
40500000
41000000
41500000
42000000
42500000
43000000
43500000
44000000
44500000
45000000
45500000
46000000
46500000
47000000
47500000
48000000
48500000
49000000
49500000
50000000
50500000
51000000
51500000
52000000
52500000
53000000
53500000
54000000
54500000
55000000
55500000
56000000
56500000
57000000
57500000
58000000
58500000
59000000
59500000
60000000
60500000
61000000
61500000
62000000
62500000
63000000
63500000
64000000
64500000
65000000
65500000
66000000
66500000
67000000
67500000
68000000
68500000
69000000
69500000
70000000
70500000
71000000
71500000
7

In [7]:
from collections import defaultdict


p3s = defaultdict(list)
t3s = defaultdict(list)

for i in range(len(mallpreds)):
    for j in range(6):
        p3 = mallpreds[i][0][j].detach().cpu().numpy()[0,0,:,:][:,:].flatten()+mallpreds[i][1][j].detach().cpu().numpy()[0,0,::-1,::-1][:,:].flatten()
        t3 = malltargets[i][0][j][0,:].reshape((250,250))[:,:].flatten()
        tn3 = malltargetnans[i][0][j][0,:].reshape((250,250))[:,:].flatten()

        valid = np.isfinite(t3) * (tn3<=0.25) 
        p3[~valid]=np.nan
        t3[~valid]=np.nan

        p3s[j].append(p3)
        t3s[j].append(t3)


In [8]:
from scipy.stats import pearsonr, spearmanr
for i in range(6):
    valid = np.isfinite(t3s[i])
    print(pearsonr(np.array(p3s[i])[valid], np.array(t3s[i])[valid])[0])

0.5141590054404548
0.574769684361197
0.6152324956081736
0.650912753330726
0.6987843835282611
0.7171001880069249
